In [1]:
%pwd

'c:\\Project\\iNeuron-DS-Assessment\\ML\\notebook'

In [4]:
import os
os.chdir("../")

In [5]:
%pwd

'c:\\Project\\iNeuron-DS-Assessment'

In [6]:
import pandas as pd

In [8]:
load_wine = pd.read_csv('ML/notebook/data\WineQT.csv')

In [11]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import randint

# Load the wine dataset
wine_data = load_wine()
X = wine_data.data
y = wine_data.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the parameter distribution for hyperparameter tuning
param_dist = {
    'criterion': ['gini', 'entropy'],
    'max_depth': randint(1, 20),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 20),
    'max_features': ['sqrt', 'log2', None]
}

# Perform randomized search cross validation
random_search = RandomizedSearchCV(DecisionTreeClassifier(), param_distributions=param_dist, n_iter=100, cv=5, random_state=42)
random_search.fit(X_train, y_train)

# Get the best decision tree model
best_dt = random_search.best_estimator_

# Make predictions on the test set
y_pred = best_dt.predict(X_test)

# Calculate the accuracy of the best model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of Decision Tree Classifier:", accuracy)


Accuracy of Decision Tree Classifier: 0.9722222222222222


In [10]:
from sklearn.ensemble import RandomForestClassifier

shuffle_split = ShuffleSplit(n_splits=10, test_size=0.2, random_state=42)

forest = []
for train_index, _ in shuffle_split.split(X_train):
    X_train_subset, y_train_subset = X_train[train_index], y_train[train_index]
    dt = DecisionTreeClassifier(**random_search.best_params_)
    dt.fit(X_train_subset, y_train_subset)
    forest.append(dt)

forest_accuracy = []
for dt in forest:
    y_pred = dt.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    forest_accuracy.append(accuracy)

average_accuracy = sum(forest_accuracy) / len(forest_accuracy)
print("Average Accuracy of Random Forest:", average_accuracy)


Average Accuracy of Random Forest: 0.913888888888889
